<a href="https://colab.research.google.com/github/HSE-LAMBDA/mldm-2019/blob/master/day-2/NoFreeLunch_seminar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Can you solve a supervised learning task with no knowledge about the nature of the data?

In [0]:
!wget https://github.com/HSE-LAMBDA/mldm-2019/raw/master/day-2/data_test.csv
!wget https://github.com/HSE-LAMBDA/mldm-2019/raw/master/day-2/data_train.csv

In [0]:
import numpy as np
import pandas as pd

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline

In [0]:
df_train = pd.read_csv('data_train.csv', index_col=None)
df_test  = pd.read_csv('data_test.csv' , index_col=None)

X_train = df_train[['x0', 'x1']].values
y_train = df_train['y'].values
X_test  = df_test [['x0', 'x1']].values
y_test  = df_test ['y'].values

In [0]:
def test_model(model, preprocess=lambda X: X):
  model = Pipeline([('scaler', RobustScaler()), ('estimator', model)])
  y_scaler = RobustScaler()
  model.fit(preprocess(X_train), y_scaler.fit_transform(y_train[:,np.newaxis]).reshape(-1))

  mse_train = mean_squared_error(y_train, y_scaler.inverse_transform(model.predict(preprocess(X_train))[:,np.newaxis] ))
  mse_test  = mean_squared_error(y_test , y_scaler.inverse_transform(model.predict(preprocess(X_test ))[:,np.newaxis] ))

  return mse_train**0.5, mse_test**0.5

In [0]:
models = [
    ('LinearRegression(),', LinearRegression()),
    ('KNeighborsRegressor(n_neighbors=1),', KNeighborsRegressor(n_neighbors=1)),
    ('KNeighborsRegressor(n_neighbors=5),', KNeighborsRegressor(n_neighbors=5)),
    ('KNeighborsRegressor(n_neighbors=10),', KNeighborsRegressor(n_neighbors=10)),
    ('RandomForestRegressor(n_estimators=50, min_samples_split=2),', RandomForestRegressor(n_estimators=50, min_samples_split=2)),
    ('RandomForestRegressor(n_estimators=50, min_samples_split=5),', RandomForestRegressor(n_estimators=50, min_samples_split=5)),
    ('RandomForestRegressor(n_estimators=50, min_samples_split=10),', RandomForestRegressor(n_estimators=50, min_samples_split=10)),
]

scores = [test_model(m) for _, m in models]

for (m, _), (score_train, score_test) in zip(models, scores):
  print(m)
  print("        RMSE (train): {:.4}".format(score_train))
  print("        RMSE (test):  {:.4}".format(score_test ))

**Can you get an RMSE below 0.01?**